In [ ]:
import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT=hate_guard
%env WANDB_LOG_MODEL='end'

In [ ]:
model_name = 'microsoft/MiniLM-L12-H384-uncased'

batch_size = 64
epochs = 1000
learning_rate = 3e-5
max_seq_length = 64
early_stopping_patience = 2
early_stopping_threshold = 0.0005

text_field_name = 'tweet'
label_field_name = 'class'
num_classes = 3

seed = 42

output_dir = './data/output'

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
id2label = {0: 'Hate Speech', 1: 'Offensive Language', 2: 'Neither'}
label2id = {v:k for k, v in id2label.items()}

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read in the CSV file
data = pd.read_csv('labeled_data.csv')

# Split the data into training, validation, and test sets
train, val_test = train_test_split(data, test_size=0.08, random_state=seed)
val, test = train_test_split(val_test, test_size=0.5, random_state=seed)

# Write each set to a separate CSV file
train.to_csv('data/input/train.csv', index=False)
val.to_csv('data/input/val.csv', index=False)
test.to_csv('data/input/test.csv', index=False)

In [ ]:
data['tweet'].apply(
    lambda s: len(s.split())).describe()

In [ ]:
data['class'].value_counts()

In [ ]:
from transformers import AutoTokenizer

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe['tweet'].values
        self.labels = dataframe['class'].values
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_seq_length,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            'attention_mask': encoding['attention_mask'],
            "labels": torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)

train_dataset = CustomDataset(train)
val_dataset = CustomDataset(val)
test_dataset = CustomDataset(test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class_weights = (1 - (data['class'].value_counts().sort_index() / len(data))).values
class_weights

In [ ]:
class_weights = torch.from_numpy(class_weights).float().to(device)
class_weights

In [ ]:
from torch import nn
from transformers import Trainer

class WeightedLossTrainer(Trainer):
    def copmpute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits = outputs.get('logits')
        labels = inputs.get('labels')
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_classes,
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    recall = recall_score(labels, preds, average=None)
    precision = precision_score(labels, preds, average=None)
    f1 = f1_score(labels, preds, average='weighted')
    
    return {
        'f1': f1,
        'accuracy': accuracy,
        'recall_HS': recall[0],
        'recall_OL': recall[1],
        'recall_N': recall[2],
        'precision_HS': precision[0],
        'precision_OL': precision[1],
        'precision_N': precision[2],
    }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=1000,
    learning_rate=learning_rate,
    weight_decay=0.01,
    logging_dir='./data/output/logs',
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    fp16=True,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy = 'epoch',
    report_to='wandb',
    run_name="hateguard-0.3",
)

In [ ]:
from transformers import EarlyStoppingCallback

early_stop = EarlyStoppingCallback(early_stopping_patience, early_stopping_threshold)

In [ ]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stop],
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
wandb.finish()

In [ ]:
trainer.save_model('./data/output/best_model')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('./data/output/best_model/tokenizer')